In [393]:
# importing the required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time

df = pd.read_csv('../Data/profile_links.csv')
df["0"]
pl = list(df["0"])


all_user = []
for i in range(1):
    one_user = []
    print("Serial :",i)
    url = pl[1]
    print(url)
    driver = webdriver.Chrome(r'C:\Users\USER\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    time.sleep(3)
    print("here")
    
    
    #basic informations
    basic = []
    source_code = driver.page_source
    soup = BeautifulSoup(source_code, 'html.parser')
    name = soup.find_all('strong', class_ = 'userName--1ZA07')
    for n in name:
        basic.append(n.text)
    sp = soup.find_all('div', class_ = 'categoryName--1zWtA')
    for m in sp:
        basic.append(m.text)
    rating = soup.find_all('div', class_ = 'itemRating--360UA itemRating--2-rFv typeLarge--1cEMN')
    for k in rating:
        basic.append(k.text)
    reviews = soup.find_all('span', class_ = 'statsNum--32OX2')
    for kk in reviews:
        basic.append(kk.text)
    print("here 1")
    
    
    ### Project data for one user
    maininfo = []
    infos = soup.find_all('ul', class_ = 'productInfoList--1-H-D')
    for f in infos:   
        li = f.find_all('li')
        for ll in li:
            uh = ["대표자","상호명","사업자등록번호","통신판매업번호-", "사업장 주소", "고객센터",'메일']
            for u in range(len(uh)):
                if uh[u] in ll.text:
                    b = uh[u]
                    la = ll.text
                    maininfo.append(la.replace(b , ""))   
    
    print("here 2")
    
    
    #count product and review section
    products = []
    tt = soup.find_all('div', class_ = "list--e6w5E")
    for t in tt:  
        cc = t.find_all('div', class_='count--2w5o6')
        for cd in cc:
            cd.find_all('div', class_ = "count--2w5o6")
            ce = cd.text
            print(ce)
            products.append(ce)
            
    print("here 3")
            
    ### Project data for one user
    
    projects = soup.find_all('div', class_ = 'listArea--peDdh')
#   projects  and consultations 
    all_project = []
    for y in projects:
        one = []
        yy = y.find_all('div', class_ = 'item--1ZJSx')
        for t in yy:
            project_item = []
            tdiv = t.find_all('div', class_ =['itemTitle--2vWBq','elip2--nFWXY'])
            for td in tdiv:
                project_title = td.text
                project_item.append(project_title)

            ratdiv = t.find_all('div', class_ =['itemGroup--2RnIL','ItemGroup_itemGroup--1f-on'])
            for rd in ratdiv:
                rating = rd.text
                project_item.append(rating)

            feediv = t.find_all('span', class_ =['priceInner--1HE2v'])
            for fd in feediv:
                fee = fd.text
                project_item.append(fee)
            one.append([*project_item])   
        all_project.append([*one])
    p = []
    for i in range(len(all_project)):
        data = all_project[i]
        for j in range(len(data)):
            dj = data[j]
            for k in range(len(dj)):
                bb = dj[k]
                p.append(bb)   
    #review object
    rdiv = soup.find_all('div', class_ = "listSection--kViCl")
    reviews_rating = []
    reviews_heading = []
    reviews_text = []

    for eachr in rdiv:
        ee = eachr.find_all('div', class_ = "reviewItem--1OwNO")
        for each in ee:
            rating = each.find_all('div', class_ = ["expertPoint--2Zrvr","expertPoint--13H3V"])
            for r in rating:
                reviews_rating.append(r.text)

            head = each.find_all('div', class_ = "reviewTitle--qv3Pk")
            for r in head:
                reviews_heading.append(r.text)

            commentdiv = each.find_all('p', class_ = "reviewText--28mzN")
            for ecom in commentdiv:
                reviews_text.append(ecom.text)

    review_obj = []
    for i in range(len(reviews_heading)):
        review_obj.append(reviews_heading[i])
        review_obj.append(reviews_rating[i])
        review_obj.append(reviews_text[i])

    #final works  
    print("here 5")
    all_user.append([url,*basic, *maininfo, *products, *p, *review_obj ])    
    driver.quit()

Serial : 0
https://m.kin.naver.com/profile/index.nhn?u=zvRvZSrTBCeYUJjRbEc5OKhzAkyWWga1TYilww4ty7Y%3D
here
here 1
here 2
3
715
here 3
here 5


In [394]:
all_user

[['https://m.kin.naver.com/profile/index.nhn?u=zvRvZSrTBCeYUJjRbEc5OKhzAkyWWga1TYilww4ty7Y%3D',
  '이재용',
  '손해배상',
  '교통사고',
  '형사사건',
  '평균 평점4.9',
  '715',
  '934',
  '이재용',
  'JY법률사무소',
  '128-32-34769',
  '',
  '(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층',
  '025824833',
  'wjdgml1278@naver.com',
  '3',
  '715',
  '신속하고 정확한 형사사건 법률상담',
  '평균 평점4.9후기391',
  '판매가10,000원할인률75%할인 전 가격40,000',
  '성범죄 상담',
  '평균 평점4.9후기232',
  '판매가10,000원할인률75%할인 전 가격40,000',
  '명쾌하고 꼼꼼한 20분 심층 전화상담',
  '평균 평점4.8후기92',
  '판매가20,000원할인률60%할인 전 가격50,000',
  '성범죄 상담',
  '평점5',
  '늦은시간에다 억울함에 말도 더듬게 되고 화가 치밀어오르는데 차분하게 이야기 들어주시고 상담해주셔서 감사합니다 정말 억울한 상황 풀고 싶습니다 감사합니다',
  '성범죄 상담',
  '평점5',
  '늦은시간에 상담감사합니다 ',
  '성범죄 상담',
  '평점5',
  '빠르게 제가 궁금한 부분에 대해서 설명해주십니다!']]

In [395]:
test = pd.DataFrame(all_user)
test

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,715,934,이재용,JY법률사무소,...,"판매가20,000원할인률60%할인 전 가격50,000",성범죄 상담,평점5,늦은시간에다 억울함에 말도 더듬게 되고 화가 치밀어오르는데 차분하게 이야기 들어주시...,성범죄 상담,평점5,늦은시간에 상담감사합니다,성범죄 상담,평점5,빠르게 제가 궁금한 부분에 대해서 설명해주십니다!


In [401]:
new = test.T
new

,0
0,https://m.kin.naver.com/profile/index.nhn?u=zv...
1,이재용
2,손해배상
3,교통사고
4,형사사건
5,평균 평점4.9
6,715
7,934
8,이재용
9,JY법률사무소


In [402]:
index = ['url',"name","specialty 1","specialty 2","specialty 3","rating","review", "consultations","Title" ,"Company" , "Comapny Regestration","Mail order Business Number","Address","Service center","Mail","Product", "Review",'Consultation 01', "Con01 Rating/Review", "Con01 Discount/Fee", 'Consultation 02', "Con02 Rating/Review", "Con02 Discount/Fee",'Consultation 03', "Con03 Rating/Review", "Con03 Discount/Fee","Review 01 Heading", "Review 01 Rating","Review 01 Text","Review 02 Heading", "Review 02 Rating","Review 02 Text","Review 03 Heading", "Review 03 Rating","Review 03 Text"  ]
len(index)

35

In [403]:
new.index = index

In [404]:
new

,0
url,https://m.kin.naver.com/profile/index.nhn?u=zv...
name,이재용
specialty 1,손해배상
specialty 2,교통사고
specialty 3,형사사건
rating,평균 평점4.9
review,715
consultations,934
Title,이재용
Company,JY법률사무소


In [259]:
test.columns = ['url',"name","specialty 1","specialty 2","specialty 3","rating","review", "consultations","Title" ,"Company" , "Comapny Regestration","Mail order Business Number","Address","Service center","Mail","Product", "Review","Works" ]

In [260]:
test

,url,name,specialty 1,specialty 2,specialty 3,rating,review,consultations,Title,Company,Comapny Regestration,Mail order Business Number,Address,Service center,Mail,Product,Review,Works
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,715,934,이재용,JY법률사무소,128-32-34769,,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,715,"[[신속하고 정확한 형사사건 법률상담, 평균 평점4.9후기391, 판매가10,000..."


In [262]:
test.drop('review',axis = 1)

,url,name,specialty 1,specialty 2,specialty 3,rating,consultations,Title,Company,Comapny Regestration,Mail order Business Number,Address,Service center,Mail,Product,Review,Works
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,934,이재용,JY법률사무소,128-32-34769,,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,715,"[[신속하고 정확한 형사사건 법률상담, 평균 평점4.9후기391, 판매가10,000..."


In [263]:
test

,url,name,specialty 1,specialty 2,specialty 3,rating,review,consultations,Title,Company,Comapny Regestration,Mail order Business Number,Address,Service center,Mail,Product,Review,Works
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,715,934,이재용,JY법률사무소,128-32-34769,,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,715,"[[신속하고 정확한 형사사건 법률상담, 평균 평점4.9후기391, 판매가10,000..."


In [281]:
f = []
works = test["Works"]
for i in works:
    for j in i:
        for m in j:
            f.append(m)
            
f

['신속하고 정확한 형사사건 법률상담',
 '평균 평점4.9후기391',
 '판매가10,000원할인률75%할인 전 가격40,000',
 '성범죄 상담',
 '평균 평점4.9후기232',
 '판매가10,000원할인률75%할인 전 가격40,000',
 '명쾌하고 꼼꼼한 20분 심층 전화상담',
 '평균 평점4.8후기92',
 '판매가20,000원할인률60%할인 전 가격50,000']

In [297]:
ff = pd.Series(f)

ff = ff.T

In [300]:
frames = [test, ff]
result = pd.concat(frames, axis = 1)
result

,url,name,specialty 1,specialty 2,specialty 3,rating,review,consultations,Title,Company,Comapny Regestration,Mail order Business Number,Address,Service center,Mail,Product,Review,Works,0
0,https://m.kin.naver.com/profile/index.nhn?u=zv...,이재용,손해배상,교통사고,형사사건,평균 평점4.9,715,934,이재용,JY법률사무소,128-32-34769,,"(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층",025824833,wjdgml1278@naver.com,3,715,"[[신속하고 정확한 형사사건 법률상담, 평균 평점4.9후기391, 판매가10,000...",신속하고 정확한 형사사건 법률상담
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,평균 평점4.9후기391
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"판매가10,000원할인률75%할인 전 가격40,000"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,성범죄 상담
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,평균 평점4.9후기232
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"판매가10,000원할인률75%할인 전 가격40,000"
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,명쾌하고 꼼꼼한 20분 심층 전화상담
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,평균 평점4.8후기92
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"판매가20,000원할인률60%할인 전 가격50,000"


In [162]:
basic

['이재용', '손해배상', '교통사고', '형사사건', '평균 평점4.9', '715', '934']

In [164]:
maininfo

['이재용',
 'JY법률사무소',
 '128-32-34769',
 '',
 '(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층',
 '025824833',
 'wjdgml1278@naver.com']

In [163]:
all_projects

[['신속하고 정확한 형사사건 법률상담', '평균 평점4.9후기391', '판매가10,000원할인률75%할인 전 가격40,000'],
 ['성범죄 상담', '평균 평점4.9후기232', '판매가10,000원할인률75%할인 전 가격40,000'],
 ['명쾌하고 꼼꼼한 20분 심층 전화상담', '평균 평점4.8후기92', '판매가20,000원할인률60%할인 전 가격50,000']]

In [205]:
products

['3', '715']

# Basic

In [168]:
basic = []

for i in range(1):
#     print("Serial :,"i)
    all_projects = []
    url = pl[1]
    print(url)
    driver = webdriver.Chrome(r'C:\Users\USER\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    time.sleep(3)
    source_code = driver.page_source
    soup = BeautifulSoup(source_code, 'html.parser')
    name = soup.find_all('strong', class_ = 'userName--1ZA07')
    for n in name:
        basic.append(n.text)
        
    category = soup.find_all('strong', class_ = 'introCategory--F81Ky')
    for e in category:
        basic.append(e.text)
        
    sp = soup.find_all('div', class_ = 'categoryName--1zWtA')
    for m in sp:
        basic.append(m.text)
    rating = soup.find_all('div', class_ = 'itemRating--360UA itemRating--2-rFv typeLarge--1cEMN')
    for k in rating:
        basic.append(k.text)
    reviews = soup.find_all('span', class_ = 'statsNum--32OX2')
    for kk in reviews:
        basic.append(kk.text) 
        

    driver.quit()
print(basic)

https://m.kin.naver.com/profile/index.nhn?u=zvRvZSrTBCeYUJjRbEc5OKhzAkyWWga1TYilww4ty7Y%3D
['이재용', '법률', '손해배상', '교통사고', '형사사건', '평균 평점4.9', '715', '934']


In [169]:
basic

['이재용', '법률', '손해배상', '교통사고', '형사사건', '평균 평점4.9', '715', '934']

# Projects

In [186]:
projects = soup.find_all('div', class_ = 'listArea--peDdh')
projects

[<div class="listArea--peDdh"><div class="item--1ZJSx"><a class="itemCard--2Whvq ItemSmallThumbnail_itemCard--196nY productCard--1WjeX" href="/mobile/expert/product/detail?storeId=100002042&amp;productId=100003754"><div class="itemLink--2ljnw"><div class="itemBox--3y657"><div class="itemThumb--Ke4MS"><div class="thumbnail--WbaVj" style='background-image: url("https://phinf.pstatic.net/image.nmv/kin_2020_09_08_5/e2161813-f183-11ea-9635-246e963a4708_01.jpg");'></div></div><div class="itemInfo--24tcX"><div class="itemTitle--2vWBq elip2--nFWXY">신속하고 정확한 형사사건 법률상담</div><div class="itemGroup--2RnIL ItemGroup_itemGroup--1f-on"><div class="itemRating--360UA"><span class="blind">평균 평점</span><svg class="iconStar--2zx5M" height="15" viewbox="0 0 13 12" width="15"><path d="M6.5 9.75l-3.82 2.009.73-4.255L.317 4.491l4.272-.62L6.5 0l1.91 3.87 4.272.621L9.59 7.504l.73 4.255z" fill="#4A65F6" fill-rule="evenodd"></path></svg>4.9</div><div class="itemCount--2HsJv"><em class="countText--3Xuau">후기</em><spa

In [189]:
### Project data for one user
projects = soup.find_all('div', class_ = 'listArea--peDdh')

all_projects = []
for y in projects:
    yy = y.find_all('div', class_ = 'item--1ZJSx')
    for t in yy:
        project_item = []
        tdiv = t.find_all('div', class_ =['itemTitle--2vWBq','elip2--nFWXY'])
        for td in tdiv:
            project_title = td.text
            project_item.append(project_title)
            
        ratdiv = t.find_all('div', class_ =['itemGroup--2RnIL','ItemGroup_itemGroup--1f-on'])
        for rd in ratdiv:
            rating = rd.text
            project_item.append(rating)
            
        feediv = t.find_all('span', class_ ='priceInner--1HE2v')
        for fd in feediv:
            fee = fd.text
            project_item.append(fee)
        
        all_projects.append(project_item)

In [188]:
all_projects

[['신속하고 정확한 형사사건 법률상담', '평균 평점4.9후기391', '판매가10,000원할인률75%할인 전 가격40,000'],
 ['성범죄 상담', '평균 평점4.9후기232', '판매가10,000원할인률75%할인 전 가격40,000'],
 ['명쾌하고 꼼꼼한 20분 심층 전화상담', '평균 평점4.8후기92', '판매가20,000원할인률60%할인 전 가격50,000']]

In [160]:
temp = []

for item in all_projects:
    for i in item:
        hh = ["평균평점","후기","판매가","원할인률","원할인률-", "할인 전 가격"]
        for yt in hh:
            if yt in i:
                temp.append(i.replace(yt ," "))

temp

['평균 평점4.9 391',
 ' 10,000원할인률75%할인 전 가격40,000',
 '판매가10,000원할인률75% 40,000',
 '평균 평점4.9 232',
 ' 10,000원할인률75%할인 전 가격40,000',
 '판매가10,000원할인률75% 40,000',
 '평균 평점4.8 92',
 ' 20,000원할인률60%할인 전 가격50,000',
 '판매가20,000원할인률60% 50,000']

In [113]:
for item in project_item:
    print("명쾌하고" in item)

명쾌하고 꼼꼼한 20분 심층 전화상담
True
평균 평점4.8후기92
False
판매가20,000원할인률60%할인 전 가격50,000
False


# product and review number

In [204]:
products = []

tt = soup.find_all('div', class_ = "list--e6w5E")
for t in tt:  
    cc = t.find_all('div', class_='count--2w5o6')
    for cd in cc:
        cd.find_all('div', class_ = "count--2w5o6")
        ce = cd.text
        print(ce)
        products.append(ce)
products

3
715


['3', '715']

# Seller main information

In [98]:
### Project data for one user
maininfo = []
infos = soup.find_all('ul', class_ = 'productInfoList--1-H-D')
for f in infos:   
    li = f.find_all('li')
    for ll in li:
        uh = ["대표자","상호명","사업자등록번호","통신판매업번호-", "사업장 주소", "고객센터",'메일']
        for u in range(len(uh)):
            if uh[u] in ll.text:
                b = uh[u]
                la = ll.text
                maininfo.append(la.replace(b , ""))   
maininfo

['이재용',
 'JY법률사무소',
 '128-32-34769',
 '',
 '(우:06647) 서울특별시 서초구 서초대로 254 (서초동, 오퓨런스) 14층',
 '025824833',
 'wjdgml1278@naver.com']

# Reviews

In [375]:
### review data for one user
rdiv = soup.find_all('div', class_ = ['titleSection--1lnfz','SubTitle_titleSection--2YQaq'])
for i in rdiv:
    ii = i.find_all('div', class_ = "titleArea--s3-rN")
    for each in ii:
        ai = each.find_all('a', class_ = "titleLink--2BV8L")
        for a in ai:
            print(a)
       

<a class="titleLink--2BV8L" href="/mobile/expert/product/list?storeId=100002042"><span class="linkText--Nm8Me">3개의 상품 더보기</span><svg class="iconCircleArrow--oYTT-" height="18" viewbox="0 0 20 20" width="18"><g fill="none" fill-rule="evenodd"><circle cx="10" cy="10" r="9.5" stroke="#DBDBDB"></circle><path d="M6.5 10h6M11 7l3 3-3 3" stroke="#7E96FF" stroke-linecap="round" stroke-linejoin="round" stroke-width="1.5"></path></g></svg></a>
<a class="titleLink--2BV8L" href="/mobile/expert/product/list?storeId=100002042"><span class="linkText--Nm8Me">더보기</span><svg class="iconCircleArrow--oYTT-" height="18" viewbox="0 0 20 20" width="18"><g fill="none" fill-rule="evenodd"><circle cx="10" cy="10" r="9.5" stroke="#DBDBDB"></circle><path d="M6.5 10h6M11 7l3 3-3 3" stroke="#7E96FF" stroke-linecap="round" stroke-linejoin="round" stroke-width="1.5"></path></g></svg></a>
<a class="titleLink--2BV8L" href="/mobile/expert/review/list?u=zvRvZSrTBCeYUJjRbEc5OKhzAkyWWga1TYilww4ty7Y%3D"><span class="linkTex

In [376]:
rdiv = soup.find_all('div', class_ = "listSection--kViCl")
reviews_rating = []
reviews_heading = []
reviews_text = []

for eachr in rdiv:
    ee = eachr.find_all('div', class_ = "reviewItem--1OwNO")
    for each in ee:
        rating = each.find_all('div', class_ = ["expertPoint--2Zrvr","expertPoint--13H3V"])
        for r in rating:
            reviews_rating.append(r.text)
            
        head = each.find_all('div', class_ = "reviewTitle--qv3Pk")
        for r in head:
            reviews_heading.append(r.text)

        commentdiv = each.find_all('p', class_ = "reviewText--28mzN")
        for ecom in commentdiv:
            reviews_text.append(ecom.text)
            
review_obj = []

for i in range(len(reviews_heading)):
    review_obj.append(reviews_heading[i])
    review_obj.append(reviews_rating[i])
    review_obj.append(reviews_text[i])


In [377]:
reviews_rating

['평점5', '평점5', '평점5']

In [378]:
reviews_heading

['성범죄 상담', '성범죄 상담', '성범죄 상담']

In [379]:
reviews_text

['늦은시간에다 억울함에 말도 더듬게 되고 화가 치밀어오르는데 차분하게 이야기 들어주시고 상담해주셔서 감사합니다 정말 억울한 상황 풀고 싶습니다 감사합니다',
 '늦은시간에 상담감사합니다 ',
 '빠르게 제가 궁금한 부분에 대해서 설명해주십니다!']

In [380]:
review_obj = []

for i in range(len(reviews_heading)):
    review_obj.append(reviews_heading[i])
    review_obj.append(reviews_rating[i])
    review_obj.append(reviews_text[i])

In [381]:
review_obj

['성범죄 상담',
 '평점5',
 '늦은시간에다 억울함에 말도 더듬게 되고 화가 치밀어오르는데 차분하게 이야기 들어주시고 상담해주셔서 감사합니다 정말 억울한 상황 풀고 싶습니다 감사합니다',
 '성범죄 상담',
 '평점5',
 '늦은시간에 상담감사합니다 ',
 '성범죄 상담',
 '평점5',
 '빠르게 제가 궁금한 부분에 대해서 설명해주십니다!']